# Evaluation of pretrained and PEFT finetuned models

## Setup

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.eval.evaluation import run_transformer_evaluation, run_adapter_evaluation

## Evaluation configuration

This section contains the list of tasks (evaluations) and models to evaluate.

In [3]:
CUSTOM_TASKS_PATH = "src.eval.custom_usmle_qa"

tasks_to_run = [
    "community|usmle-qa-letter|1|0",
    "community|usmle-qa-text|1|0",
    "community|usmle-qa-letter-text|1|0",
    "community|usmle-qa-mcf|4|0",
    "community|usmle-qa-cf|4|0",
]

# Output location
gcs_bucket_name = "open-llm-finetuning"
output_directory = f"gcs://{gcs_bucket_name}/evaluation_results"

# Define transformer-based models to evaluate
transformer_models_to_eval = [
        {
            "model_name": "meta-llama/Llama-2-7b-hf",
            "use_chat_template": False,
            "batch_size": 4,
        },
        {
            "model_name": "meta-llama/Meta-Llama-3-8B",
            "use_chat_template": False,
            "batch_size": 1,
        },
        {
            "model_name": "Sirius27/BeingWell_llama2_7b",
            "use_chat_template": False,
            "batch_size": 4,
        },
        {
            "model_name": "johnsnowlabs/JSL-MedLlama-3-8B-v1.0",
            "use_chat_template": False,
            "batch_size": 2,
        },
    ]

# Define adapter-based (fine-tuned) models to evaluate
adapter_models_to_eval = [
        {
            "model_name": "pippalap/llama3-8b-usmle-prefix-letters",
            "base_model": "meta-llama/Meta-Llama-3-8B",
            "adapter_weights": True,
            "use_chat_template": False,
            "batch_size": 1,
            "tokenizer_name": "meta-llama/Meta-Llama-3-8B",
        },
        {
            "model_name": "jihbr/usmle-llama8b-dora-letters-v1",
            "adapter_weights": True,
            "base_model": "meta-llama/Meta-Llama-3-8B",
            "use_chat_template": False,
            "batch_size": 1,
            "tokenizer_name": "meta-llama/Meta-Llama-3-8B",
        },
        {
            "model_name": "jihbr/usmle-llama8b-qlora_letters",
            "adapter_weights": True,
            "base_model": "meta-llama/Meta-Llama-3-8B",
            "use_chat_template": False,
            "batch_size": 1,
            "tokenizer_name": "meta-llama/Meta-Llama-3-8B",
        },
    ]

## Run evaluations

This section contains the loops to execute the evaluation runs.

In [ ]:
# Loop through each task and run evaluations
for task in tasks_to_run:
    print(f"===== Running Task: {task} =====")

    # Evaluate adapter models
    for model_details in adapter_models_to_eval:
        run_adapter_evaluation(
            adapter_weights=model_details["adapter_weights"],
            model_name=model_details["model_name"],
            base_model=model_details["base_model"],
            tasks=task,
            batch_size=model_details["batch_size"],
            output_dir=output_directory,
            use_chat_template=model_details["use_chat_template"],
            custom_tasks_path=CUSTOM_TASKS_PATH,
            tokenizer_name=model_details.get("tokenizer_name")
        )
    
    # Evaluate transformer models
    for model_details in transformer_models_to_eval:
        run_transformer_evaluation(
            model_name=model_details["model_name"],
            tasks=task,
            batch_size=model_details["batch_size"],
            output_dir=output_directory,
            use_chat_template=model_details["use_chat_template"],
            custom_tasks_path=CUSTOM_TASKS_PATH,
            tokenizer_name=model_details.get("tokenizer_name")
        )
print("===== All Evaluations Complete =====")

===== Running Task: community|usmle-qa-letter-text|1|0 =====
Starting adapter evaluation for adapter: pippalap/llama3-8b-usmle-prefix-letters
On base model: meta-llama/Meta-Llama-3-8B
Tasks: community|usmle-qa-letter-text|1|0
Output Directory: gcs://open-llm-finetuning/evaluation_results


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

If you want to use extended_tasks, make sure you installed their dependencies using `pip install -e .[extended_tasks]`.
You cannot select the number of dataset splits for a generative evaluation at the moment. Automatically inferring.
Greedy generation:   0%|          | 0/1273 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/peft/peft_model.py:2060: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")

Greedy generation:  35%|███▌      | 446/1273 [03:31<04:40,  2.95it/s]